In [18]:
#importing useful libraries
import re
import numpy as np
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.stats import t
from sklearn.manifold import TSNE


In [19]:
#import processed data
with open('plot_time.csv', 'r') as init_data:
    raw_data = pd.read_csv(init_data)

In [20]:
#a little data preparation
patients = raw_data.loc[:,"RID"].unique()

changers = np.zeros(len(patients))
impaired = np.zeros(len(patients))

cata = pd.DataFrame(patients)
cata["change"] = changers
cata["impaired"] = impaired
cata.rename(columns={0:"RID"}, inplace=True)

for i,v in enumerate(cata.loc[:,"RID"]):
    vals = raw_data.loc[raw_data["RID"] == v, ["DX_bl", "DXCHANGE"]]
    if sum(vals.DXCHANGE > 3) > 0:
        cata.loc[i, "change"] = 1
    if ((vals.DX_bl.any() != "CN") | sum(vals.DXCHANGE > 1) > 0):
        cata.loc[i, "impaired"] = 1      
merge = pd.merge(raw_data, cata, on="RID", how="outer")   
merge.PTGENDER = merge.PTGENDER.astype("category")
merge.PTMARRY = merge.PTMARRY.astype("category")
merge.PTETHCAT = merge.PTETHCAT.astype("category")
merge.PTRACCAT = merge.PTRACCAT.astype("category")
merge.change = merge.change.astype("category")
merge.impaired = merge.impaired.astype("category")


In [21]:
filled = merge.copy()
fill4 = filled.fillna(filled.groupby('RID').transform('mean'))

In [22]:
#we only want to do the statistices on the feature columns identified in the exploratory data analysis

features = [4, 5, 10, 12, 13, 15, 16, 17, 21, 22, 24, 25, 26, 27, 35, 37, 43, 44, 45]
target = 48

In [23]:
x = fill4.iloc[:,features]
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12741 entries, 0 to 12740
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   AGE                         12741 non-null  float64 
 1   PTGENDER                    12741 non-null  category
 2   APOE4                       12729 non-null  float64 
 3   CDRSB_bl                    12741 non-null  float64 
 4   ADAS11_bl                   12724 non-null  float64 
 5   MMSE_bl                     12741 non-null  int64   
 6   RAVLT_immediate_bl          12705 non-null  float64 
 7   RAVLT_learning_bl           12705 non-null  float64 
 8   Ventricles_bl               12251 non-null  float64 
 9   Hippocampus_bl              10960 non-null  float64 
 10  Entorhinal_bl               10847 non-null  float64 
 11  Fusiform_bl                 10847 non-null  float64 
 12  MidTemp_bl                  10847 non-null  float64 
 13  ICV_bl          

In [24]:
#more data preparation, in this case converting categorical text fields to integers
t1 = x.copy()
t1["rid"] =  filled.RID
t1["impaired"] = filled.impaired
t1["female"] = 1
t1.loc[t1.PTGENDER == "Male","female"] = 0
t1 = t1.drop("PTGENDER", axis=1)
t1.impaired = t1.impaired.astype("int")
t2 = t1.dropna()
t2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7302 entries, 13 to 12740
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AGE                         7302 non-null   float64
 1   APOE4                       7302 non-null   float64
 2   CDRSB_bl                    7302 non-null   float64
 3   ADAS11_bl                   7302 non-null   float64
 4   MMSE_bl                     7302 non-null   int64  
 5   RAVLT_immediate_bl          7302 non-null   float64
 6   RAVLT_learning_bl           7302 non-null   float64
 7   Ventricles_bl               7302 non-null   float64
 8   Hippocampus_bl              7302 non-null   float64
 9   Entorhinal_bl               7302 non-null   float64
 10  Fusiform_bl                 7302 non-null   float64
 11  MidTemp_bl                  7302 non-null   float64
 12  ICV_bl                      7302 non-null   float64
 13  MMSE                        730

In [35]:
#A comparison of the patients who were diagnosed with cognitive impairment at any point in the trial against those that were cognitively normal throughout.

ts = t2.groupby("impaired")
ts.mean()

,AGE,APOE4,CDRSB_bl,ADAS11_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,Ventricles_bl,Hippocampus_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MMSE,Hippocampus,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,rid,female
impaired,,,,,,,,,,,,,,,,,,,,
0,74.214137,0.225681,0.021077,5.636868,29.083009,46.095331,6.047341,32035.066796,7444.773671,3866.231518,17979.483787,20227.112192,1.503678e+06,29.047656,7310.269021,1352.630141,230.083902,20.976990,2423.945525,0.523346
1,72.519878,0.607465,1.632726,10.522092,27.315278,34.928993,4.185764,38634.225174,6783.648785,3498.003472,17510.220486,19598.831250,1.537185e+06,26.191679,6606.066772,999.563654,300.087753,28.951714,2628.656250,0.432986


In [45]:
#APOE4 is ordinal data
#female and impaired are nominal
#MMSE_bl and MMSE are count data
#hence we will do chi tests on APOE4, female and impaired
t3 = t2.copy()
categoricals = ["APOE4", "female", "impaired"]
for x in categoricals:
    t3[x] = t3.loc[:,x].astype("category")
cats = t3.select_dtypes(include=["category"], exclude=None)
print(cats.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 7302 entries, 13 to 12740
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   APOE4     7302 non-null   category
 1   impaired  7302 non-null   category
 2   female    7302 non-null   category
dtypes: category(3)
memory usage: 78.7 KB
None


In [52]:
#use this version of the function in reporting
def chi_tester_p(df, ind, target):
    from scipy.stats import chi2_contingency
    from scipy.stats import chi2
# contingency table
    table = pd.crosstab(df.loc[:,ind], df.loc[:,target])
    #print("-----------table----------")
    #print(table)
    stat, p, dof, expected = chi2_contingency(table)

    return(p)

In [53]:
#this version gives the readout of the chi test
def chi_tester(df, ind, target):
    from scipy.stats import chi2_contingency
    from scipy.stats import chi2
# contingency table
    table = pd.crosstab(df.loc[:,ind], df.loc[:,target])
    print("-----------table----------")
    print(table)
    stat, p, dof, expected = chi2_contingency(table)
    print('dof=%d' % dof)
    print(expected)
# interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
# interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    print("P is:", p)
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')

In [68]:
"""for i in t2.columns:
    plt.subplots(figsize=(12,6))
    plt.subplot(1,2,1,)
    sns.boxplot(y=t2[i], x = t2.impaired, data = t2)
    plt.title(str(i) + " by cognitive class")
    plt.subplot(1,2,2)
    if t2[i].dtype == ("float64" or "int64"):
        sns.distplot(t2[i], kde=False )
        plt.title("Overall Distribution")
        
    else:
        sns.countplot(x=i, data = t2)
        
    plt.show()"""
cols = list(ts.get_group(0).columns)   
for c,v in enumerate(cols):
    m = ts.get_group(0)[v].mean() - ts.get_group(1)[v].mean()
    tango = scipy.stats.ttest_ind(ts.get_group(0)[v], ts.get_group(1)[v], axis=0, equal_var=False, nan_policy='propagate')
    
    #mango = scipy.stats.chisquare(f_obs=ts.get_group(1)[v], f_exp=ts.get_group(0)[v])
    line_new = '{} {:<8} {} {:<+9.3} {} {:<.4}'.format("t-test on:", v, "difference in means:", m, "p values is:", tango.pvalue)
    print("\n------------------------------------")
    if v in list(cats.columns):
        mango = chi_tester_p(cats, "impaired", v)
        line_chi = '{} {:<} {} {:<.4}'.format("chi-test on:", v, "p value is:", mango)
        print(line_chi)
        if mango < 0.05:
            print("There is a significant relationship between impaired status and", v)
        else:
            print("No significant relationship between feature", v, "and the impairment status")
    else:
        print(line_new)
        if tango.pvalue < 0.05:
            print("There is a significant relationship between impaired status and", v)
        else:
            print("No significant relationship between feature", v, "and the impairment status")
    
    
    plt.subplots(figsize=(12,6))
    plt.subplot(1,2,1,)
    sns.boxplot(y=t2[v], x = t2.impaired, data = t2)
    plt.title(str(v) + " by cognitive class")
    plt.subplot(1,2,2)
    if t2[v].dtype == ("float64" or "int64"):
        sns.distplot(t2[v], kde=False )
        plt.title("Overall Distribution")
        
    else:
        sns.countplot(x=v, data = t2)
        
    plt.show()
    

TypeError: 'NoneType' object is not iterable

In [51]:
# number of patients (not data points) in each group
ts.rid.nunique()

impaired
0    187
1    799
Name: rid, dtype: int64

In [46]:
chis = list(cats.columns)
chis.remove("impaired")
for x in chis:
    chi_tester(cats, "impaired", x)
    print("--------------------\n")

-----------table----------
APOE4      0.0   1.0  2.0
impaired                 
0         1216   304   22
1         2940  2141  679
dof=2
[[ 877.64338537  516.32292523  148.0336894 ]
 [3278.35661463 1928.67707477  552.9663106 ]]
probability=0.950, critical=5.991, stat=412.083
Dependent (reject H0)
significance=0.050, p=0.000
P is: 3.2915502067364575e-90
Dependent (reject H0)
--------------------

-----------table----------
female       0     1
impaired            
0          735   807
1         3266  2494
dof=1
[[ 844.91125719  697.08874281]
 [3156.08874281 2603.91125719]]
probability=0.950, critical=3.841, stat=39.731
Dependent (reject H0)
significance=0.050, p=0.000
P is: 2.914749440518602e-10
Dependent (reject H0)
--------------------



In [31]:
pd.crosstab(cats.female, [cats.impaired, cats.APOE4])

impaired    0              1           
APOE4     0.0  1.0 2.0   0.0   1.0  2.0
female                                 
0         556  165  14  1715  1134  417
1         660  139   8  1225  1007  262

In [32]:
stat, p, dof, expected = scipy.stats.chi2_contingency(pd.crosstab(cats.impaired, cats.APOE4))
print(stat, p, dof)
print(pd.crosstab(cats.impaired, cats.APOE4))
print(expected)

412.0825994564058 3.2915502067364575e-90 2
APOE4      0.0   1.0  2.0
impaired                 
0         1216   304   22
1         2940  2141  679
[[ 877.64338537  516.32292523  148.0336894 ]
 [3278.35661463 1928.67707477  552.9663106 ]]


In [33]:
# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
# contingency table
table = [[10, 20, 30],[6,  9,  17]]
print(table)
stat, p, dof, expected = chi2_contingency(table)
print('dof=%d' % dof)
print(expected)
# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

[[10, 20, 30], [6, 9, 17]]
dof=2
[[10.43478261 18.91304348 30.65217391]
 [ 5.56521739 10.08695652 16.34782609]]
probability=0.950, critical=5.991, stat=0.272
Independent (fail to reject H0)
significance=0.050, p=0.873
Independent (fail to reject H0)


In [34]:

from scipy.stats import chi2_contingency
from scipy.stats import chi2
# contingency table
table = pd.crosstab(cats.impaired, cats.APOE4)
print(table)
stat, p, dof, expected = chi2_contingency(table)
print('dof=%d' % dof)
print(expected)
# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

APOE4      0.0   1.0  2.0
impaired                 
0         1216   304   22
1         2940  2141  679
dof=2
[[ 877.64338537  516.32292523  148.0336894 ]
 [3278.35661463 1928.67707477  552.9663106 ]]
probability=0.950, critical=5.991, stat=412.083
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
